# Run ECG-LUX RealTime
1 - Import bitalino, numpy, pylab and time python libraries

In [1]:
import bitalino
import numpy as np
import time
from pylab import *
%matplotlib inline

2 - Input your device's MAC address and wait 1 second

In [ ]:
#Set MAC (Windows)
macAddress = "20:17:09:18:46:98"
device = bitalino.BITalino(macAddress)
time.sleep(1)

3 - Set parameters for the biosignal acquisition:
     Sampling rate (srate), Number of frames (nframes)

In [3]:
#Biosignal acquisition parameters
srate = 100
nframes = 300

4 - Set parameters for signal processing

In [4]:
#Processing parameters
luxthreshold = 10
winsize = 300
drange = np.arange(winsize)
slwin = 3
pwinrange = 10
pwin = np.arange(-pwinrange,pwinrange,1)
hbfactor = 2.5
p2pfactor = 5
bpmthreshold = 70

5 - Define smooth moving average function

In [5]:
#Moving average
def slideMean(x, N):
    return np.convolve(x, np.ones((N,))/N)[(N-1):]

6 - Start  a continuous acquisiton using only ECG and LUX analog channels

In [ ]:
device.start(srate, [1,5])
print ("START")

7 - Get the first seconds warmup, run an acquistion reading nframes and perform the following actions:

    a. Stop when pressing the button for a while
    b. Check the light level, if bright:
        b1. Launch LED light sequence
        b2. Estimate beats per minute (bpm) and apply threshold
        b3. If greater than bpm threshold set alarm (longer LED sequence / buzzer / pwm)

In [ ]:
#Warmup data collection (initialise)
data = device.read(nframes)
ECG0=[]
ECG0 = np.append(ECG0,data[:, -2])
data = device.read(nframes)
ECG0 = np.append(ECG0,data[:, -2])
data = device.read(nframes)
ECG0 = np.append(ECG0,data[:, -2])

#Loop execution
ECG = ECG0
try:
    while True:

        #Read data
        data = device.read(nframes)
        
        #Check button
        if np.mean(data[:, 1]) < 1: break

        LUX = data[:, -1]
        ECG = np.append(ECG[nframes:],data[:, -2])        
        
        #Thresholding LUX average 
        luxmean = np.mean(abs((LUX)))
        if luxmean < luxthreshold:
            #Starting LED sequence
            device.trigger([1, 0])
            device.trigger([0, 0])
            device.trigger([1, 0])
            device.trigger([0, 0])

            #Estimate hr
            hrd = -1 
            conv = slideMean(ECG,slwin)
            lconv = len(conv)
            conv = conv[slwin:lconv-slwin]
            ECG_absdiff = abs(conv)
                
            ### First derivative
            conv1 = np.diff(conv)
            
            ### Second derivative
            conv2 = np.diff(conv1)
            data = conv2
            
            #Get peaks using second derivative and std
            dstd = np.std(data)
            peaks = np.where(abs(data) > hbfactor*dstd)[0]
            
            #Avoid neighbours pointing to the same peak
            dpeaks = np.diff(peaks)
            ipeaks = np.append(True, (dpeaks > 20))
            rpeaks = peaks[ipeaks]
            rpeaks0 = rpeaks
                
            #Peak to peak range to confirm
            is_peak = np.zeros((len(rpeaks)), dtype=bool)
            for j in xrange(0,len(rpeaks)):
                pwin_i = pwin + rpeaks[j]
                pwin_i = np.intersect1d(pwin_i[(pwin_i >= 0)],pwin_i[(pwin_i < len(data))])
                p2p = np.ptp(data[pwin_i])
                if (p2p > p2pfactor*dstd):
                    is_peak[j]=True      
            rpeaks=rpeaks[is_peak]
                
            #Compute hr(bpm)
            s0 = size(rpeaks0) 
            s = size(rpeaks)
            if s > 1:
                hrd = (rpeaks[-1]-rpeaks[0])
                hrt = 1.0/(s-1) * hrd * 1.0/srate
                hr = 60.0 / hrt
                if hr > bpmthreshold:
                    #Alarm ==> BUZ + Conclude LED sequence (longer)
                    device.trigger([1, 1])
                    device.trigger([0, 0])
                    device.trigger([1, 0])
                    device.trigger([0, 0])
                    device.trigger([1, 0])
                    device.trigger([0, 0])
                    
                else:
                    #No Alarm ==> Conclude LED sequence
                    device.trigger([1, 0])
                    device.trigger([0, 0])
                    
            else:
                hr = -1

        else:
            #Turn LED off
            device.trigger([0, 0])
finally:
    device.trigger([0, 0])
    print "STOP"
    device.stop()

8 - Close device

In [ ]:
device.close()

In [ ]:
data = loadtxt("ECG_LUX_handson.txt")

In [ ]:
ECG = np.array(data[:,5])
LUX = np.array(data[:,6])
fig = plt.figure()
ax = fig.add_subplot(111)
fig.subplots_adjust(top=0.85)
ax.set_title('Raw ECG signal')
ax.set_xlabel('Sample')
ax.set_ylabel('Val')
plot(ECG[drange],'#d21959')
